# Overview Materi

Source: https://www.youtube.com/watch?v=LDRbO9a6XPU

Jelaskan secara singkat apa itu decision tree menurut pemahamanmu!

# Import Data & Libraries

In [1]:
from __future__ import print_function

# label kolom
header = ["color", "diameter", "label"]

# data training
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# data testing
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# Fungsi Dasar

In [2]:
# fungsi mencari apa saja unique value dari suatu kolom
def unique_values(data, col_index):
    """
    Mengembalikan list berisi nilai unik dari kolom tertentu.

    Parameters:
    - data: list of list (dataset)
    - col_index: int (index kolom yang ingin dicari nilai uniknya)

    Returns:
    - list berisi nilai unik
    """
    values = set()
    for row in data:
        values.add(row[col_index])
    return list(values)


# contoh penggunaan
print(unique_values(training_data, 0))

['Red', 'Green', 'Yellow']


In [3]:
# fungsi Menghitung jumlah unique value dari suatu kolom
def class_counts(rows):
    """
    Menghitung jumlah masing-masing nilai unik (label) di dataset.

    Parameters:
    - rows: list of list (dataset)

    Returns:
    - counts: dict, key = nilai unik, value = jumlah kemunculan
    """
    counts = {}
    for row in rows:
        # asumsikan label ada di kolom terakhir
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts


# contoh penggunaan
print(class_counts(training_data))

{'Apple': 2, 'Grape': 2, 'Lemon': 1}


In [4]:
# fungsi pengecekan suatu value numerik atau bukan
def is_numeric(value):
    """
    Mengecek apakah value bersifat numerik (int atau float).

    Parameters:
    - value: objek apapun

    Returns:
    - True jika numerik, False jika bukan
    """
    return isinstance(value, int) or isinstance(value, float)


# contoh penggunaan
print(is_numeric(10))
print(is_numeric(3.14))
print(is_numeric("Apple"))

True
True
False


In [5]:
class Question:

    # inisialisasi kolom dan nilai pertanyaan
    def __init__(self, column, value):
        self.column = column  # index kolom
        self.value = value    # nilai pertanyaan

    # mengecek apakah contoh data sesuai dengan pertanyaan
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    # menampilkan pertanyaan dalam format string yang mudah dibaca
    def __repr__(self):
        condition = ">=" if is_numeric(self.value) else "=="
        return f"Is column[{self.column}] {condition} {self.value}?"


# contoh penggunaan
q1 = Question(0, 'Red')
q2 = Question(1, 3)

print(q1)
print(q2.match(['Green', 4, 'Apple']))

Is column[0] == Red?
True


In [6]:
# membagi dataset menjadi dua berdasarkan pertanyaan
def partition(rows, question):
    """
    Membagi dataset menjadi dua bagian:
    - true_rows: baris yang memenuhi pertanyaan
    - false_rows: baris yang tidak memenuhi pertanyaan
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows


# contoh penggunaan
q = Question(0, 'Red')
true_rows, false_rows = partition(training_data, q)

print("True rows:")
for row in true_rows:
    print(row)

print("\nFalse rows:")
for row in false_rows:
    print(row)

True rows:
['Red', 1, 'Grape']
['Red', 1, 'Grape']

False rows:
['Green', 3, 'Apple']
['Yellow', 3, 'Apple']
['Yellow', 3, 'Lemon']


**apa itu gini impurity?**
<br> gini impurity berfungsi mengukur tingkat ketidakmurnian atau ketidakteraturan pada sebuah simpul (node) dalam pohon

In [7]:
# menghitung nilai Gini Impurity untuk sebuah dataset
def gini(rows):
    """
    Menghitung Gini Impurity untuk dataset.

    Gini Impurity = 1 - sum(p_i^2)
    dimana p_i adalah proporsi tiap label
    """
    counts = class_counts(rows)
    impurity = 1
    total = len(rows)
    for lbl in counts:
        prob_of_lbl = counts[lbl] / total
        impurity -= prob_of_lbl ** 2
    return impurity

# contoh penggunaan
print(gini(training_data))

q = Question(0, 'Red')
true_rows, false_rows = partition(training_data, q)
print("Gini True Rows:", gini(true_rows))
print("Gini False Rows:", gini(false_rows))


0.6399999999999999
Gini True Rows: 0.0
Gini False Rows: 0.4444444444444445


**apa itu information gain?**
<br> information gain berfungsi mengukur seberapa efektif sebuah fitur dalam memisahkan data berdasarkan kelas-kelasnya

In [8]:
# menghitung nilai Information Gain dari pemisahan dataset
def info_gain(left, right, current_uncertainty):
    """
    Menghitung Information Gain dari pemisahan dataset menjadi dua bagian.

    Parameters:
    - left: list, baris yang memenuhi pertanyaan
    - right: list, baris yang tidak memenuhi pertanyaan
    - current_uncertainty: float, Gini impurity sebelum split

    Returns:
    - float, Information Gain
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

# contoh penggunaan
current_uncertainty = gini(training_data)

q = Question(0, 'Red')
true_rows, false_rows = partition(training_data, q)

gain = info_gain(true_rows, false_rows, current_uncertainty)
print("Information Gain:", gain)


Information Gain: 0.37333333333333324


In [9]:
# mencari pertanyaan terbaik untuk membagi dataset berdasarkan information gain tertinggi
def find_best_split(rows):
    """
    Mencari pertanyaan terbaik untuk membagi dataset
    berdasarkan Information Gain tertinggi.
    """
    best_gain = 0  # nilai Information Gain tertinggi
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # kolom terakhir adalah label

    for col in range(n_features):
        # dapatkan semua nilai unik di kolom ini
        values = set(row[col] for row in rows)

        for val in values:
            question = Question(col, val)

            # splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # hitung Information Gain
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # update best gain dan best question
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

# contoh penggunaan
best_gain, best_question = find_best_split(training_data)
print("Best Gain:", best_gain)
print("Best Question:", best_question)


Best Gain: 0.37333333333333324
Best Question: Is column[1] >= 3?


# Fungsi Decision Tree

In [10]:
# merepresentasikan node daun (leaf) pada decision tree yang berisi hasil prediksi
class Leaf:

    # inisialisasi leaf dengan menghitung jumlah kemunculan tiap kelas
    def __init__(self, rows):
        self.predictions = class_counts(rows)

leaf = Leaf(training_data)
print(leaf.predictions)


{'Apple': 2, 'Grape': 2, 'Lemon': 1}


In [11]:
# merepresentasikan node keputusan (decision node) yang berisi pertanyaan dan cabang
class Decision_Node:

    # inisialisasi node dengan pertanyaan, cabang benar, dan cabang salah
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = Question
        self.true_branch = true_branch
        self.false_branch = false_branch



In [12]:
# membangun decision tree secara rekursif
def build_tree(rows):
    gain, question = find_best_split(rows)

    if gain == 0:
        return Leaf(rows)

    true_rows, false_rows = partition(rows, question)

    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)

    return Decision_Node(question, true_branch, false_branch)


In [13]:
# mencetak struktur decision tree secara rekursif dalam format teks
def print_tree(node, spacing=""):

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return

    # mencetak pertanyaan pada node saat ini
    print(spacing + str(node.question))

    # mencetak cabang true secara rekursif
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # mencetak cabang false secara rekursif
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

# contoh penggunaan
my_tree = build_tree(training_data)
print_tree(my_tree)


<class '__main__.Question'>
--> True:
  <class '__main__.Question'>
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


In [19]:
# mengklasifikasikan satu baris data menggunakan decision tree
def classify(row, node):
    if isinstance(node, Leaf):
        # kembalikan label dengan jumlah terbanyak
        return max(node.predictions, key=node.predictions.get)
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)



In [20]:
# menampilkan prediksi pada leaf dalam format persentase
def print_leaf(counts):
    total = sum(counts.values())
    probs = {}
    for lbl in counts:
        probs[lbl] = f"{int(counts[lbl] / total * 100)}%"
    return probs

# contoh penggunaan
leaf = Leaf(training_data)
probs = print_leaf(leaf.predictions)
print("Prediksi persentase:", probs)


Prediksi persentase: {'Apple': '40%', 'Grape': '40%', 'Lemon': '20%'}


# Predict Using Decision Tree

In [ ]:
# menguji decision tree dengan data uji dan membandingkan hasil prediksi dengan label asli
for row in testing_data:
    prediction_counts = classify(row, my_tree)
    prediction = print_leaf(prediction_counts)
    print("Data:", row, "Prediksi:", prediction, "Label asli:", row[-1])
